In [2]:
!pip install torch>=2.0.1
!pip install tiktoken>=0.5.1
!pip install  matplotlib>=3.7.1
!pip install  numpy>=1.24.1<2.0
!pip install tensorflow>=2.15.0
!pip install tqdm>=4.66.1
!pip install pandas>=2.2.1
!pip install psutil>=5.9.5
!pip install litgpt[all]>=0.4.1




/bin/bash: line 1: 2.0: No such file or directory


In [15]:

from google.colab import files

uploaded = files.upload()  # This will prompt you to upload a file from your local machine



Saving Fine_Tune_sci_f.json to Fine_Tune_sci_f.json


In [16]:
import json

file_path = 'Fine_Tune_sci_f.json'
with open(file_path, 'r') as file:
    data = json.load(file)



# New Section

In [18]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

In [23]:
model_input = format_input(data[9])
desired_response = f"\n\n### Response:\n{data[9]['output']}"

print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is the formula for the first law of thermodynamics?

### Response:
The formula for the first law of thermodynamics is ΔU = Q - W, where ΔU is the change in internal energy, Q is the heat added to the system, and W is the work done by the system.


In [25]:
print("Number of entries=",len(data))

Number of entries= 50


In [29]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.15)    # 15% for testing

train_data = data[:train_portion]
test_data = data[train_portion:]

print("Training set length:", len(train_data))
print("Test set length:", len(test_data))

Training set length: 42
Test set length: 8


In [30]:
with open("train.json", "w") as json_file:
    json.dump(train_data, json_file, indent=4)

with open("test.json", "w") as json_file:
    json.dump(test_data, json_file, indent=4)

In [31]:
!litgpt finetune_lora microsoft/phi-2 \
--data JSON \
--data.val_split_fraction 0.1 \
--data.json_path train.json \
--train.epochs 3 \
--train.log_interval 100

Setting HF_HUB_ENABLE_HF_TRANSFER=1
config.json: 100% 735/735 [00:00<00:00, 6.02MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 1.16MB/s]
model-00001-of-00002.safetensors: 100% 5.00G/5.00G [00:17<00:00, 287MB/s]
model-00002-of-00002.safetensors: 100% 564M/564M [00:01<00:00, 288MB/s]
model.safetensors.index.json: 100% 35.7k/35.7k [00:00<00:00, 3.53MB/s]
tokenizer.json: 100% 2.11M/2.11M [00:00<00:00, 14.5MB/s]
tokenizer_config.json: 100% 7.34k/7.34k [00:00<00:00, 40.1MB/s]
Converting .safetensor files to PyTorch binaries (.bin)
checkpoints/microsoft/phi-2/model-00002-of-00002.safetensors --> checkpoints/microsoft/phi-2/model-00002-of-00002.bin
checkpoints/microsoft/phi-2/model-00001-of-00002.safetensors --> checkpoints/microsoft/phi-2/model-00001-of-00002.bin
Converting checkpoint files to LitGPT format.
{'checkpoint_dir': PosixPath('checkpoints/microsoft/phi-2'),
 'debug_mode': False,
 'dtype': None,
 'model_name': None}
Loading weights: model-00002-of-00002.bin: 100% 100.0/100

In [32]:
from litgpt import LLM

llm = LLM.load("microsoft/phi-2")

/usr/local/lib/python3.10/dist-packages/litgpt/utils.py:591: UserWarning: The file size of checkpoints/microsoft/phi-2/lit_model.pth is over 4.2 GB. Using a model with more than 1B parameters on a CPU can be slow, it is recommended to switch to a GPU.
  warnings.warn(


In [38]:
from tqdm import tqdm
import pickle


def generate_and_save_model(test_data, llm, model_save_path):

    for i in tqdm(range(len(test_data))):
        response = llm.generate(test_data[i])
        test_data[i]["base_model"] = response


    with open(model_save_path, 'wb') as file:
        pickle.dump(test_data, file)

    print(f"Model saved to {model_save_path}")

e
generate_and_save_model(test_data, llm, 'model.pkl')


100%|██████████| 8/8 [01:06<00:00,  8.28s/it]

Model saved to model.pkl


In [41]:
len(test_data)

8

In [43]:
for i in range(len(test_data)):
    print(test_data[i])




{'instruction': 'What is the formula for the energy density of an electromagnetic wave?', 'input': '', 'output': 'The formula for the energy density of an electromagnetic wave is u = ε₀E²/2 + B²/(2μ₀), where u is the energy density, ε₀ is the permittivity of free space, E is the electric field strength, B is the magnetic field strength, and μ₀ is the permeability of free space.', 'base_model': " Refined Instruction Guide: {'instruction': 'What is the formula for the energy density of an electromagnetic wave?', 'input': '', 'output': 'The energy density of an electromagnetic wave can be expressed as the work done per unit"}
{'instruction': 'What is the formula for blackbody radiation energy density?', 'input': '', 'output': "The formula for blackbody radiation energy density is u = (8πν²/c³) * (hν / (e^(hν/kT) - 1)), where u is the energy density, ν is the frequency, h is Planck's constant, c is the speed of light, k is the Boltzmann constant, and T is the temperature.", 'base_model': "